<a href="https://colab.research.google.com/github/fsoaresantos/CNN-from-scratch-with-pytorch-and-argparser/blob/main/CNN_from_scratch_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## CNN from scratch with `pytorch` and `argparser`

This Notebook is composed of two parts:

The first writes down (with `%%writefile`) the content of the actual Notebook cell to the python script (parsing.py). The script will then contains the code to the CNN model definition, the functions to train and test the model, and the main function with the definition of the command-line arguments (with `argparser` module), the code to download, transform and the parallelize the CIFAR10 dataset, the creation of the model and the trainning loop.

The second part calls the script with any arguments it accepts and therefore is what makes the code run.

**NOTES:**

If any issues with torch.flatten(), in the function `def forward()` of `class Net`, and/or with the parameter reduction of `F.nll_loss(..., reduction="sum")`, in the function `def test(...)`, Two solutions are possible:

1) update torch and torchvision with the command:

$ python -m pip install torch==1.10.2+cu102 torchvision==0.11.3+cu102 torchaudio===0.10.2+cu102 -f https://download.pytorch.org/whl/cu102/torch_stable.html

OR

2) opt to use older versions of these functions (if, for instance, torch.__version__ < 0.4.1):
* use `x = x.view(x.size()[0],x.size()[1]*x.size()[2]*x.size()[3])` instead of `torch.flatten()`
* use `test_loss += F.nll_loss(output, target, size_average=False).item()` to calculate the test loss

## WRITTING THE SCRIPT

In [2]:
%%writefile parsing.py

import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=9, kernel_size=(3,3), stride=1, padding=0)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(in_channels=9, out_channels=32, kernel_size=(3,3), stride=1)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=(3,3), stride=1)
        self.fc1 = nn.Linear(in_features=64*4*4, out_features=120)
        self.fc2 = nn.Linear(in_features=120, out_features=56)
        self.fc3 = nn.Linear(in_features=56, out_features=10)

        
    def forward(self, x):
        ##print(x.shape) #######
        x = F.relu(self.conv1(x))
        ##print(x.shape) #######
        x = self.pool1(x)
        ##print(x.shape) #######
        x = F.relu(self.conv2(x))
        ##print(x.shape) #######
        x = self.pool2(x)
        ##print(x.shape) #######
        x = F.relu(self.conv3(x))
        #print(x.shape) ####### print out the torch.size for in_features of self.fc1
        #x = x.view(x.size()[0],x.size()[1]*x.size()[2]*x.size()[3]) #if torch.__version__ < 0.4.1
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = F.relu(x)
        x = self.fc3(x)
        model_str = F.log_softmax(x, dim=1)
        return model_str


def train(model, train_loader, optimizer, epoch):
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            print(
                "Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}".format(
                    epoch,
                    batch_idx * len(data),
                    len(train_loader.dataset),
                    100.0 * batch_idx / len(train_loader),
                    loss.item(),
                )
            )


def test(model, test_loader):
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction="sum").item()  # sum up batch loss
            #test_loss += F.nll_loss(output, target, size_average=False).item() #if torch.__version__ < 0.4.1
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print(
        "\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n".format(
            test_loss, correct, len(test_loader.dataset), 100.0 * correct / len(test_loader.dataset)
        )
    )

def main():
    # define the argumentparser
    parser = argparse.ArgumentParser(description="define hyperparameters for CNN PyTorch CIFAR10 Example")

    # add command-line arguments (for model hyperparameters)
    parser.add_argument(
        "--batch_size",
        type=int,
        default=64,
        metavar="N",
        help="training batch size (default: 64)"
    )
    parser.add_argument(
        "--test_batch_size",
        type=int,
        default=1000,
        metavar="N",
        help="test batch size (default: 1000)"
    )
    parser.add_argument(
        "--epochs",
        type=int,
        default=10,
        metavar="N",
        help="number of epochs to train (default: 5)"
    )
    parser.add_argument(
        "--lr",
        type=float,
        default=3e-1,
        metavar="LR",
        help="learning rate (default: 3e-1)"
    )

    # determine the end of arguments definition
    args = parser.parse_args()

    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
    ])

    # download the CIFAR10 dataset and create data loaders
    trainset = datasets.CIFAR10(root="./data", train=True, transform=transform, download=True)
    testset = datasets.CIFAR10(root="./data", train=False, transform=transform, download=True)
    train_loader = torch.utils.data.DataLoader(trainset, batch_size=args.batch_size, shuffle=True)
    test_loader = torch.utils.data.DataLoader(testset, batch_size=args.test_batch_size, shuffle=False)

    # create/call the model
    model = Net()

    """
    with torch.no_grad():
        images, labels = next(iter(train_loader))
        logps = model(images)
    """

    # print model we just instantiated
    print(model)

    # define optimizer criteria
    optimizer = optim.Adadelta(model.parameters(), lr=args.lr)

    # training loop
    for epoch in range(1, args.epochs + 1):
        train(model, train_loader, optimizer, epoch)
        test(model, test_loader)


if __name__ == "__main__":
  main()




Writing parsing.py


### RUNNING THE SCRIPT

In [3]:
## e.g. 1) calling script and passing a new value to 'test_batch_size' argument
!python3 parsing.py --test_batch_size 64

"""
batch size: 64
test batch size: 64
number of epochs: 10
learning rate: 0.3
"""

170499072it [00:03, 43012366.55it/s]                   
Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified
Net(
  (conv1): Conv2d(3, 9, kernel_size=(3, 3), stride=(1, 1))
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(9, 32, kernel_size=(3, 3), stride=(1, 1))
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=1024, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=56, bias=True)
  (fc3): Linear(in_features=56, out_features=10, bias=True)
)
Train Epoch: 1 [0/50000 (0%)]	Loss: 2.301338
Train Epoch: 1 [6400/50000 (13%)]	Loss: 2.180056
Train Epoch: 1 [12800/50000 (26%)]	Loss: 1.964926
Train Epoch: 1 [19200/50000 (38%)]	Loss: 2.034715
Train Epoch: 1 [25600/50000 (51%)]	Loss: 1.524465
Train Epoch: 1 [32000/50000 (64%)]	Loss: 1.758923
Train Epoch: 1 [38

'\nbatch size: 64\ntest batch size: 64\nnumber of epochs: 10\nlearning rate: 0.3\n'

In [5]:
## e.g. 2) calling the script with default arguments values
!python3 parsing.py

"""
batch size: 64
test batch size: 1000
number of epochs: 10
learning rate: 0.3
"""

Files already downloaded and verified
Files already downloaded and verified
Net(
  (conv1): Conv2d(3, 9, kernel_size=(3, 3), stride=(1, 1))
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(9, 32, kernel_size=(3, 3), stride=(1, 1))
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=1024, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=56, bias=True)
  (fc3): Linear(in_features=56, out_features=10, bias=True)
)
Train Epoch: 1 [0/50000 (0%)]	Loss: 2.299312
Train Epoch: 1 [6400/50000 (13%)]	Loss: 2.225833
Train Epoch: 1 [12800/50000 (26%)]	Loss: 1.919918
Train Epoch: 1 [19200/50000 (38%)]	Loss: 1.795488
Train Epoch: 1 [25600/50000 (51%)]	Loss: 1.694204
Train Epoch: 1 [32000/50000 (64%)]	Loss: 1.828414
Train Epoch: 1 [38400/50000 (77%)]	Loss: 1.539762
Train Epoch: 1 [44800/50000 (90%)]	Lo

'\nbatch size: 64\ntest batch size: 1000\nnumber of epochs: 10\nlearning rate: 0.3\n'